In [21]:
import sys
import glob
import base64

from collections import Counter
from typing import List, Tuple

# 1. encode hex string to base64

In [2]:
my_str = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
my_str_as_bytes = str.encode(my_str)
print(type(my_str_as_bytes)) # ensure it is byte representation

<class 'bytes'>


In [7]:
from base64 import b64encode, b64decode

# hex -> base64
b64 = b64encode(bytes.fromhex(my_str)).decode()
print('in base64:', b64)

in base64: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


# 2. when fed 1c0111001f010100061a024b53535009181c after hex-decoding XOR'ed against 686974207468652062756c6c277320657965

In [19]:
from operator import xor


hexval1 = '1c0111001f010100061a024b53535009181c'
hexval2 = '686974207468652062756c6c277320657965'

bin_value1 = bin(int(hexval1, 16))[2:]
bin_value2 = bin(int(hexval2, 16))[2:]

desired_length = len(bin_value1) if len(bin_value1) > len(bin_value2) else len(bin_value2)
bin_value1 = bin_value1.zfill(desired_length)
bin_value2 = bin_value2.zfill(desired_length)
#step 4
result = [int(bit1) ^ int(bit2) for bit1,bit2 in zip(bin_value1,bin_value2)]
string_result = "".join([str(bits) for bits in result])
#step 5
final_output = hex(int(string_result, 2))[2:]
print(final_output)

746865206b696420646f6e277420706c6179


# 3. decipher hex encoded string: 1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736

In [23]:
def single_byte_xor(text: bytes, key: int) -> bytes:
    """Given a plain text `text` as bytes and an encryption key `key` as a byte
    in range [0, 256) the function encrypts the text by performing
    XOR of all the bytes and the `key` and returns the resultant.
    """
    return bytes([b ^ key for b in text])

In [29]:
occurance_english = {
    'a': 8.2389258,    'b': 1.5051398,    'c': 2.8065007,    'd': 4.2904556,
    'e': 12.813865,    'f': 2.2476217,    'g': 2.0327458,    'h': 6.1476691,
    'i': 6.1476691,    'j': 0.1543474,    'k': 0.7787989,    'l': 4.0604477,
    'm': 2.4271893,    'n': 6.8084376,    'o': 7.5731132,    'p': 1.9459884,
    'q': 0.0958366,    'r': 6.0397268,    's': 6.3827211,    't': 9.1357551,
    'u': 2.7822893,    'v': 0.9866131,    'w': 2.3807842,    'x': 0.1513210,
    'y': 1.9913847,    'z': 0.0746517
}

dist_english = list(occurance_english.values())

def compute_fitting_quotient(text: bytes, plot=False, title=None) -> float:
    """Given the stream of bytes `text` the function computes the fitting
    quotient of the letter frequency distribution for `text` with the
    letter frequency distribution of the English language.

    The function returns the average of absolute difference between the
    frequencies (in percentage) of letters in `text` and the corresponding
    letter in the English Language.
    """
    counter = Counter(text)
    dist_text = [
        (counter.get(ord(ch), 0) * 100) / len(text)
        for ch in occurance_english
    ]

    if plot:
        plot_linears(dist_english, dist_text, title=title)

    return sum([abs(a - b) for a, b in zip(dist_english, dist_text)]) / len(dist_text)

In [41]:
def decipher(text: bytes, plot=False) -> Tuple[bytes, int]:
    """The function deciphers an encrypted text using Single Byte XOR and returns
    the original plain text message and the encryption key.
    """
    original_text, encryption_key, min_fq = None, None, None
    for k in range(256):
        # we generate the plain text using key `k`
        _text = single_byte_xor(text, k)
        
        # we compute the fitting quotient for this generated plain text
        _fq = compute_fitting_quotient(_text, plot=plot, title=f"Key: {k}")
        
        # if the generated fitting quotient is lesser than the min_fq we've seen
        # then update the key and plain_text.
        if min_fq is None or _fq < min_fq:
            encryption_key, original_text, min_fq = k, _text, _fq

    # return the plain_text and the key with the minimum fitting quotient.
    return original_text, encryption_key, min_fq

In [32]:
hex_str = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

decipher(bytes.fromhex(hex_str))[0]

b"Cooking MC's like a pound of bacon"

In [38]:
single_byte_xor(str.encode('nerd'),123)

b'\x15\x1e\t\x1f'

# 4. Detect single-character XOR

In [42]:
with open('4.txt') as f:
    lines = f.readlines()
    
lines = [line.rstrip('\n') for line in lines]

In [50]:
import pandas as pd
import numpy as np

res = pd.DataFrame()

for i, line in enumerate(lines):
    df = pd.DataFrame(index=[i], columns=['encoded string','original text','encrypt key', 'min freq'])
    original_text, encryption_key, min_fq = decipher(bytes.fromhex(line))
    df['encoded string'] = line
    df['original text'] = original_text
    df['encrypt key'] = encryption_key
    df['min freq'] = min_fq
    res = pd.concat([res,df], axis=0)
    
best_match = res['min freq'].min()

print(res[res['min freq']==best_match])

                                        encoded string  \
170  7b5a4215415d544115415d5015455447414c155c46155f...   

                          original text  encrypt key  min freq  
170  b'Now that the party is jumping\n'           53  2.296475  


# 5. Implement repeating-key XOR

In [1]:
from binascii import hexlify

def repeating_key_xor(key, string):
    # i is the position within the key
    i = 0
    arr = []
    for ch in string:
        # Convert the key char and the plaintext char to
        # integers using `ord`, XOR them and add them to
        # the array.
        arr.append(ord(ch) ^ ord(key[i]))
        
        # Manage the "repeating" part of the repeating key.
        # If the end of the key is reached start back at the
        # beginning.
        i += 1
        if (i == len(key)):
            i = 0

    # Finally convert our array to a byte array (which
    # hexlify likes), then convert to hex and return it.
    return hexlify(bytearray(arr))

string = "Burning 'em, if you ain't quick and nimble I go crazy when I hear a cymbal"
key = 'ICE'

encrypted = repeating_key_xor(key, string)
print(encrypted)

b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20690a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'
